# Simple Example
This Jupyter notebook runs on Colab and shows a simple example of Tooling selection using Top-K and Double Round Robin

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

### **Import top_k and double_round_robin libraries**

In [ ]:
!git clone https://github.com/generative-computing/mellea-contribs.git

In [ ]:
%cd mellea-contribs
!pip install -e . -qq

## **Defining sample set of tools**

In [ ]:
from mellea import start_session
import mellea

# Start a Mellea session
m = start_session()

# Define a sample set of tools
TOOLS = [
    {
        "name": "EnterpriseScheduler",
        "short_description": "Internal tool for scheduling enterprise-wide meetings and events.",
        "long_description": """EnterpriseScheduler is a proprietary tool used for scheduling meetings, events, and resource bookings.
        It integrates with internal calendars, room booking systems, and employee availability databases."""
    },
    {
        "name": "BudgetOptimizer",
        "short_description": "Internal utility for budget forecasting and cost optimization.",
        "long_description": """BudgetOptimizer is a proprietary tool for financial planning, budget forecasting, and cost optimization.
        It integrates with internal ERP systems and provides real-time financial insights."""
    },
    {
        "name": "DocSearchPro",
        "short_description": "Internal search engine for company documents and knowledge base.",
        "long_description": """DocSearchPro is a proprietary search engine for indexing and retrieving internal documents,
        knowledge base articles, and project files. It supports advanced search queries and filters.
        """
    },
    {
        "name": "ResourceAllocator",
        "short_description": "Internal tool for resource allocation and project management.",
        "long_description": """
        ResourceAllocator is a proprietary tool for managing project resources, including team allocation,
        equipment booking, and project timelines. It integrates with internal HR and project management systems.
        """
    }
]

USER_QUERY = "Schedule a meeting for tomorrow and calculate the event budget"

## **Step 1: Top-K Tool Shortlisting**

In [ ]:
from mellea_contribs.tools.top_k import top_k
from pydantic import RootModel
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

def shortlist_tools(m, user_query, tools, k):
    return top_k(
        items=tools,
        comparison_prompt=f"""
        Select the tools most relevant to:
        '{user_query}'
        """,
        m=m,
        k=k,
    )

## **Step 2: High-Confidence Ranking with Double Round Robin**

In [ ]:
from mellea_contribs.tools.double_round_robin import double_round_robin

def rank_tools(m, user_query, tools):
    return double_round_robin(
        items=tools,
        comparison_prompt=f"""
        Given the user query:
        '{user_query}'
        select which tool is more useful.
        """,
        m=m,
    )


## **Final Tool selection**

In [ ]:
from pydantic import RootModel
from mellea.stdlib.requirement import req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

class ToolChoice(RootModel[str]):
    pass

def choose_final_tool(
    m: mellea.MelleaSession,
    user_query: str,
    ranked_tools: list[dict],
):
    tool_names = [t["name"] for t in ranked_tools]

    response = m.instruct(
        f"""
        Given the user query:
        '{user_query}'

        Choose the single best tool from this list:
        {tool_names}
        """,
        requirements=[
            req(
                "Tool must be one of the provided candidates",
                validation_fn=simple_validate(lambda s: s in tool_names),
            ),
            req("Return only the tool name"),
        ],
        strategy=RejectionSamplingStrategy(loop_budget=3),
        format=ToolChoice,
    )

    return response.value


In [ ]:
m = start_session()

top_k_tools = shortlist_tools(m, USER_QUERY, TOOLS, k=2)
ranked = rank_tools(m, USER_QUERY, top_k_tools)
final_tool = choose_final_tool(m, USER_QUERY, [t for t, _ in ranked])

print("Selected tool:", final_tool)